<a href="https://colab.research.google.com/github/Aasthapaudel/-Imagine-cup-cloud-skill-Challenge-/blob/main/sectrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2 in Google Colab
> 🗣️ Large Language Model Course

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne), based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da). Special thanks to Tolga HOŞGÖR for his solution to empty the VRAM.

This notebook runs on a T4 GPU. (Last update: 01 Aug 2023)


In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curr

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "Hemg/nepaligpt-llama3-8b"

# The instruction dataset to use
dataset_name = "teksingh/NagarGPT-dataset0"

# Fine-tuned model name
new_model = "NagarGPT"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
from datasets import Dataset
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/46.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/352 [00:00<?, ? examples/s]

In [7]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['मानव'])):
        text = f"### Question: {example['मानव'][i]}\n ### Answer: {example['सहायक'][i]}"
        output_texts.append(text)
    return output_texts

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# EOS_TOKEN = '<|endoftext|>'
# # Example data creation to simulate your dataset
# data = {
#     'chosen': [
#         'मानव: डायनासोरले कस्तो आवाज निकाले?\n\nसहायक: मानव र डायनासोरहरू एकै समयमा बाँच्दैनन्, त्यसैले यो भन्न गाह्रो छ। डायनासोरले के आवाज निकाल्छ भनेर पत्ता लगाउनको लागि उत्तम ठाउँ\n\n'
#     ]
# }
# #dataset = Dataset.from_dict(data)

# # Function to split the 'chosen' field and format according to the template
# formatted_data = []
# for index, row in df.iterrows():

#     question = row['मानव']
#     answer = row['सहायक']
#     formatted_text = f"### प्रश्न:\n{question}\n\n### जवाफ:\n{answer}""" + EOS_TOKEN
#     formatted_data.append({'text': formatted_text})


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [12]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [13]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['मानव'])):
        text = f"### Question: {example['मानव'][i]}\n ### Answer: {example['सहायक'][i]}"
        output_texts.append(text)
    return output_texts
x=formatting_prompts_func(dataset)

In [14]:
x

['### Question: जन्मदर्ता गर्नको लागी आवश्यक कागजातहरु के के चाहिन्छ\n ### Answer: जन्मदर्ताको लागी आवश्यक कागजपत्रहरु यस प्रकारका छन्| \n१.हजुरले public.donidcr.gov.np प्रयोग गरी अनलाईन फाराम भरेर लिइएको टोकन नम्बर चाहिन्छ,\n२.आफ्नो आमा बाब्रु को नागरिकता माणपत्रको प्रतिलिपि लिनुपर्ने हुन्छ,\n३.यसको साथै आमाबाबुको विवाह प्रमाणपत्रको प्रतिलिपि पनि आवश्यक पर्छ,\n४.अस्पतालमा जन्म भएकाको हकमा अस्पतालबाट जारी जन्म प्रमाणपत्र लिनुपर्ने हुन्छ,\n५.यदि घरमा जन्म भएको भए खोपकार्ड वा अन्य जन्ममिति खुल्ने कागजातहरु आवश्यक पर्छ,\n६.यदी हजुर बसाईँसराईं गरी आएको भए बसाइँसराईं प्रमाणपत्रको प्रतिलिपि पनि लिनुपर्नेहुन्छ,\n७.तर विदेशमा जन्म भएकाको हकमा सम्बन्धित नेपाली नियोगबाट वा कन्सुलर सेवा विभागबाट प्रमाणित कागजपत्र पनि चाहिन्छ,|',
 '### Question: जन्मदर्ताको सेवा शुल्क तथा दस्तुर रकम के कति लाग्छ\n ### Answer: जन्मदर्ता संबन्धित शुल्कको विवरण तल दिएको छ|,\n१.जन्म भएको ३५ दिन भित्रमा दर्ता गर्दा शुल्क लाग्दैन,\n२.तर ३५ दिनपछि भने रू.५० शुल्क तिर्नुपर्ने हुन्छ |,',
 '### Question: जन्मदर्ता गर्न कति 

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import torch
from transformers import BitsAndBytesConfig


In [16]:
# Set compute dtype for 4-bit quantization
bnb_4bit_compute_dtype = "float16"  # Could be "float16" or "bfloat16" if supported
compute_dtype = getattr(torch, bnb_4bit_compute_dtype, torch.float16)  # Default to float16 if not found


In [17]:
use_4bit = True  # Set to True if you want 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",  # Could be "nf4" or "fp4"
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True  # Enable nested quantization if needed
)


In [19]:
pip install transformers bitsandbytes torch


In [21]:
pip install --upgrade transformers bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [3]:
pip install 'accelerate>=0.26.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [5]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
pip show accelerate


Name: accelerate
Version: 1.1.0.dev0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Define the model name
model_name = "Hemg/nepaligpt-llama3-8b"

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,    # Use float16 for better compatibility on supported GPUs
    bnb_4bit_use_double_quant=True           # Improves quantization performance if supported
)

# Load the model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token  # Set the padding token
    tokenizer.padding_side = "right"           # Ensure padding on the right side

    # Load the model with quantization and device configuration
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",                     # Auto-detect GPU if available
        trust_remote_code=True,                # Enable for custom code in the model repo
        low_cpu_mem_usage=True                 # Optimize for lower CPU memory usage
    )
    model.config.use_cache = False   # Disable caching for more efficient memory use in fp16/4-bit modes
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")


Error loading model or tokenizer: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`


In [ ]:
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

In [ ]:
# # Empty VRAM
# del model
# del pipe
# del trainer
# import gc
# gc.collect()
# gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)